#### Roll No : 18CS71P06
#### Assignment 3

In [1]:
import nltk
import re
import numpy as np
import mxnet as mx
import sys, os
from mxnet.contrib import text
filename = sys.argv[0]
os.path.abspath(filename+"/..")
np.random.seed(42)
mx.random.seed(42)
'''
First job is to clean and preprocess the social media text. (5)

1) Replace URLs and mentions (i.e strings which are preceeded with @)
2) Segment #hastags 
3) Remove emoticons and other unicode characters
'''

def preprocess_tweet(input_text):
    cleaned_text= input_text.lower()
    cleaned_text = re.sub(r'[^\w\s]','',cleaned_text)
    cleaned_text = re.sub(r"http\S+", "", cleaned_text)
    cleaned_text = re.sub(r"@\S+", "", cleaned_text)
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    cleaned_text = emoji_pattern.sub(r'', cleaned_text)
    cleaned_text = cleaned_text.replace("#", "")
    cleaned_text = re.sub("([a-z])([A-Z])","\g<1> \g<2>",cleaned_text)
        #print(input_text)
    return cleaned_text

/home/sukanya/anaconda3/envs/mxnet/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
file=open('cancer_data.tsv')
pos_data=[]
neg_data=[]

for line in file:
    line=line.strip().split('\t')
    #print(line[0])
    text2= preprocess_tweet(line[0]).strip().split()
    if line[1]=='yes':
        pos_data.append(text2)
    if line[1]=='no':
        neg_data.append(text2)

#print(len(pos_data), len(neg_data))
sentences= list(pos_data)
sentences.extend(neg_data)
pos_labels= [1 for _ in pos_data]
neg_labels= [0 for _ in neg_data]
y=list(pos_labels)
y.extend(neg_labels)
y=np.array(y)

    #if line[1]=='yes':
    #    pos_data.append(text2)
    #if line[1]=='no':
     #   neg_data.append(text2)

#print(len(pos_data), len(neg_data))

In [3]:
def create_word_vectors(sentences):
    '''
    Input: List of sentences
    Output: List of word vectors corresponding to each sentence, vocabulary
    '''
    padding_word = "" 
    sequence_length = max(len(x) for x in sentences)
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
    #print(padded_sentences[0])    
    seen, result = set(), []
    for item in padded_sentences:
        for word in item:
            #if word not in seen:
            seen.add(word)
            result.append(word)
    #print(result)
    wordfreq = {}
    d={}
    for word in result:
        if word not in wordfreq:
             wordfreq[word] = 0 
        wordfreq[word] += 1
    #l = wordfreq.items()
    d = sorted(wordfreq.items(), key=lambda x:x[1], reverse=True)
    #l.sort(key = lambda item: item[1])
    #wordfreq = [result.count(p) for p in result]
    #print(dict(zip(result,wordfreq)))
    #keylist = wordfreq.keys()
    #keylist.sort(reverse=true)
    vocabulary_inv = [x[0] for x in d]
    #print(d)
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    word_vectors = np.array([
            [vocabulary[word] for word in sentence]
            for sentence in padded_sentences])
    return word_vectors, vocabulary, padded_sentences
#create_word_vectors(sentences)

x, vocabulary, padded_sentences = create_word_vectors(sentences)
vocab_size=len(vocabulary)

In [4]:
def create_shuffle(x,y):
    '''
    Create an equal distribution of the positive and negative examples. 
    Please do not change this particular shuffling method.
    '''
    pos_len= len(pos_data)
    neg_len= len(neg_data)
    pos_len_train= int(0.8*pos_len)
    neg_len_train= int(0.8*neg_len)
    train_data= [(x[i],y[i]) for i in range(0, pos_len_train)]
    train_data.extend([(x[i],y[i]) for i in range(pos_len, pos_len+ neg_len_train )])
    test_data=[(x[i],y[i]) for i in range(pos_len_train, pos_len)]
    test_data.extend([(x[i],y[i]) for i in range(pos_len+ neg_len_train, len(x) )])
    
    np.random.shuffle(train_data)
    x_train=[i[0] for i in train_data]
    y_train=[i[1] for i in train_data]
    np.random.shuffle(test_data)
    x_test=[i[0] for i in test_data]
    y_test=[i[1] for i in test_data]
    
    x_train=np.array(x_train)
    y_train=np.array(y_train)
    x_test= np.array(x_test)
    y_test= np.array(y_test)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test= create_shuffle(x,y)
sentence_size = x_train.shape[1]

In [5]:
batch_size = 20
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label
num_embed = 200 # dimensions to embed words into
print('embedding dimensions', num_embed)

embed_layer = mx.sym.Embedding(data=input_x, input_dim=vocab_size, output_dim=num_embed, name='vocab_embed')

# reshape embedded data for next layer
conv_input = mx.sym.Reshape(data=embed_layer, shape=(batch_size, 1, sentence_size, num_embed))
filter_list=[2,3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, shape=(batch_size, total_filters))

num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_pool, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm


batch size 20
embedding dimensions 200
convolution filters [2, 3, 4, 5]


In [6]:
from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on, use GPU if available
ctx = mx.cpu()
arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Uniform(0.1)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']

cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

In [7]:
optimizer = 'rmsprop'
max_grad_norm = 0.01
learning_rate = 0.005
epoch = 10
predicted=[]
actual=[]
max_act=[]
print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_train.shape[0], batch_size):
        batchX = x_train[begin:begin+batch_size]
        batchY = y_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=True)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_test.shape[0], batch_size):
        batchX = x_test[begin:begin+batch_size]
        batchY = y_test[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        predicted.append(cnn_model.cnn_exec.outputs[0].asnumpy())
        actual.append(batchY)
        max_act.append(np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)

    dev_acc = num_correct * 100 / float(num_total)
    print('Without using fastText Embeddings:Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))
    #epoch.end.callback = mx.callback.early.stop(10)

optimizer rmsprop
maximum gradient 0.01
learning rate (step size) 0.005
epochs to train for 10
Without using fastText Embeddings:Iter [0] Train: Time: 19.464s, Training Accuracy: 86.500             --- Dev Accuracy thus far: 87.333
Without using fastText Embeddings:Iter [1] Train: Time: 12.501s, Training Accuracy: 89.667             --- Dev Accuracy thus far: 88.000
Without using fastText Embeddings:Iter [2] Train: Time: 16.442s, Training Accuracy: 94.667             --- Dev Accuracy thus far: 87.333
Without using fastText Embeddings:Iter [3] Train: Time: 12.525s, Training Accuracy: 97.250             --- Dev Accuracy thus far: 88.000
Without using fastText Embeddings:Iter [4] Train: Time: 12.715s, Training Accuracy: 98.417             --- Dev Accuracy thus far: 90.667
Without using fastText Embeddings:Iter [5] Train: Time: 15.118s, Training Accuracy: 99.333             --- Dev Accuracy thus far: 89.333
Without using fastText Embeddings:Iter [6] Train: Time: 15.928s, Training Accuracy:

In [8]:
t=[]
j=[]
l=[]
for pred in predicted:
    for i in pred:
        t.append(i)
for act in actual:
    for i in act:
        j.append(i)
for maxx in max_act:
    for i in maxx:
        l.append(i)
eval_metrics_2 = mx.metric.F1()
eval_metrics_2.update(labels = mx.nd.array(j), preds = mx.nd.array(t))
print(eval_metrics_2.get())

('f1', 0.45608108108108103)


#### F1-score gives us correct view of the classification. Since, the dataset is skewed towards negative than positive, hence though accuracy is high, its actually because of how accuracy is actually calculated where we consider Accuracy=(TP + TN) / (TP+TN+FP+FN), hence if even true positives are 0, TN can dominate. This is a well known case of "Accuracy Paradox" Thus we use F1 score= (1/precision)+(1/recall)"F1-score gives us correct view of the classification. 

#### Using Gradient normalization value 0f 0.5

In [9]:
batch_size = 20
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label
num_embed = 200 # dimensions to embed words into
print('embedding dimensions', num_embed)

embed_layer = mx.sym.Embedding(data=input_x, input_dim=vocab_size, output_dim=num_embed, name='vocab_embed')

# reshape embedded data for next layer
conv_input = mx.sym.Reshape(data=embed_layer, shape=(batch_size, 1, sentence_size, num_embed))
filter_list=[2,3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, shape=(batch_size, total_filters))

num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_pool, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm

batch size 20
embedding dimensions 200
convolution filters [2, 3, 4, 5]


In [10]:
from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on, use GPU if available
ctx = mx.cpu()
arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Uniform(0.1)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']

cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

In [11]:
optimizer = 'rmsprop'
max_grad_norm = 0.5
learning_rate = 0.005
epoch = 10
predicted=[]
actual=[]
max_act=[]
print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_train.shape[0], batch_size):
        batchX = x_train[begin:begin+batch_size]
        batchY = y_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=True)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_test.shape[0], batch_size):
        batchX = x_test[begin:begin+batch_size]
        batchY = y_test[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        predicted.append(cnn_model.cnn_exec.outputs[0].asnumpy())
        actual.append(batchY)
        max_act.append(np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))

    dev_acc = num_correct * 100 / float(num_total)
    print('Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))
    #epoch.end.callback = mx.callback.early.stop(10)

optimizer rmsprop
maximum gradient 0.5
learning rate (step size) 0.005
epochs to train for 10
Iter [0] Train: Time: 12.666s, Training Accuracy: 85.750             --- Dev Accuracy thus far: 90.333
Iter [1] Train: Time: 12.651s, Training Accuracy: 96.417             --- Dev Accuracy thus far: 91.667
Iter [2] Train: Time: 12.724s, Training Accuracy: 99.000             --- Dev Accuracy thus far: 90.000
Iter [3] Train: Time: 12.361s, Training Accuracy: 99.417             --- Dev Accuracy thus far: 90.667
Iter [4] Train: Time: 9.742s, Training Accuracy: 99.833             --- Dev Accuracy thus far: 90.000
Iter [5] Train: Time: 9.770s, Training Accuracy: 99.750             --- Dev Accuracy thus far: 90.000
Iter [6] Train: Time: 9.739s, Training Accuracy: 99.833             --- Dev Accuracy thus far: 89.333
Iter [7] Train: Time: 9.753s, Training Accuracy: 99.917             --- Dev Accuracy thus far: 90.667
Iter [8] Train: Time: 9.790s, Training Accuracy: 99.833             --- Dev Accuracy t

In [12]:
t=[]
j=[]
l=[]
for pred in predicted:
    for i in pred:
        t.append(i)
for act in actual:
    for i in act:
        j.append(i)
for maxx in max_act:
    for i in maxx:
        l.append(i)
eval_metrics_2 = mx.metric.F1()
eval_metrics_2.update(labels = mx.nd.array(j), preds = mx.nd.array(t))
print("f1 score on Gradient Normalization of 0.5:",eval_metrics_2.get())

f1 score on Gradient Normalization of 0.5: ('f1', 0.5618320610687023)


#### Adding Dropout layer

In [15]:
batch_size = 20
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label
num_embed = 200 # dimensions to embed words into
print('embedding dimensions', num_embed)

embed_layer = mx.sym.Embedding(data=input_x, input_dim=vocab_size, output_dim=num_embed, name='vocab_embed')

# reshape embedded data for next layer
conv_input = mx.sym.Reshape(data=embed_layer, shape=(batch_size, 1, sentence_size, num_embed))
filter_list=[2,3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, shape=(batch_size, total_filters))



batch size 20
embedding dimensions 200
convolution filters [2, 3, 4, 5]


In [16]:
dropout = 0.5
#print('dropout probability', dropout)

if dropout > 0.0:
    h_drop = mx.sym.Dropout(data=h_pool, p=dropout)
else:
    h_drop = h_pool

    num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_pool, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm

In [17]:
from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on, use GPU if available
ctx = mx.cpu()
arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Uniform(0.1)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']

cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

In [18]:
optimizer = 'rmsprop'
max_grad_norm = 0.01
learning_rate = 0.005
epoch = 10
predicted=[]
actual=[]
max_act=[]
print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_train.shape[0], batch_size):
        batchX = x_train[begin:begin+batch_size]
        batchY = y_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=True)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_test.shape[0], batch_size):
        batchX = x_test[begin:begin+batch_size]
        batchY = y_test[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        predicted.append(cnn_model.cnn_exec.outputs[0].asnumpy())
        actual.append(batchY)
        max_act.append(np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))

    dev_acc = num_correct * 100 / float(num_total)
    print('Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))

optimizer rmsprop
maximum gradient 0.01
learning rate (step size) 0.005
epochs to train for 10
Iter [0] Train: Time: 12.600s, Training Accuracy: 85.417             --- Dev Accuracy thus far: 87.333
Iter [1] Train: Time: 11.501s, Training Accuracy: 89.583             --- Dev Accuracy thus far: 88.000
Iter [2] Train: Time: 11.804s, Training Accuracy: 94.833             --- Dev Accuracy thus far: 88.667
Iter [3] Train: Time: 13.753s, Training Accuracy: 97.083             --- Dev Accuracy thus far: 89.333
Iter [4] Train: Time: 13.704s, Training Accuracy: 98.583             --- Dev Accuracy thus far: 90.667
Iter [5] Train: Time: 10.590s, Training Accuracy: 99.250             --- Dev Accuracy thus far: 89.667
Iter [6] Train: Time: 10.244s, Training Accuracy: 99.583             --- Dev Accuracy thus far: 89.333
Iter [7] Train: Time: 10.088s, Training Accuracy: 99.583             --- Dev Accuracy thus far: 89.667
Iter [8] Train: Time: 11.303s, Training Accuracy: 99.667             --- Dev Accu

In [19]:
t=[]
j=[]
l=[]
for pred in predicted:
    for i in pred:
        t.append(i)
for act in actual:
    for i in act:
        j.append(i)
for maxx in max_act:
    for i in maxx:
        l.append(i)
eval_metrics_2 = mx.metric.F1()
eval_metrics_2.update(labels = mx.nd.array(j), preds = mx.nd.array(t))
print("f1 score after using Dropout:",eval_metrics_2.get())

f1 score after using Dropout: ('f1', 0.4375)


#### Doing Xavier Initialization of the parameters 

In [20]:
batch_size = 20
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label
num_embed = 200 # dimensions to embed words into
print('embedding dimensions', num_embed)

embed_layer = mx.sym.Embedding(data=input_x, input_dim=vocab_size, output_dim=num_embed, name='vocab_embed')

# reshape embedded data for next layer
conv_input = mx.sym.Reshape(data=embed_layer, shape=(batch_size, 1, sentence_size, num_embed))
filter_list=[2,3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, shape=(batch_size, total_filters))

num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_pool, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm

batch size 20
embedding dimensions 200
convolution filters [2, 3, 4, 5]


In [21]:
from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on, use GPU if available
ctx = mx.cpu()
arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Xavier(rnd_type='uniform', factor_type='avg', magnitude=3)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']

cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

In [22]:
optimizer = 'rmsprop'
max_grad_norm = 0.01
learning_rate = 0.005
epoch = 10
predicted=[]
actual=[]
max_act=[]
print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_train.shape[0], batch_size):
        batchX = x_train[begin:begin+batch_size]
        batchY = y_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=True)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_test.shape[0], batch_size):
        batchX = x_test[begin:begin+batch_size]
        batchY = y_test[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        predicted.append(cnn_model.cnn_exec.outputs[0].asnumpy())
        actual.append(batchY)
        max_act.append(np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))

    dev_acc = num_correct * 100 / float(num_total)
    print('Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))

optimizer rmsprop
maximum gradient 0.01
learning rate (step size) 0.005
epochs to train for 10
Iter [0] Train: Time: 10.373s, Training Accuracy: 85.333             --- Dev Accuracy thus far: 86.000
Iter [1] Train: Time: 9.484s, Training Accuracy: 86.333             --- Dev Accuracy thus far: 87.667
Iter [2] Train: Time: 9.558s, Training Accuracy: 89.583             --- Dev Accuracy thus far: 85.667
Iter [3] Train: Time: 10.065s, Training Accuracy: 92.250             --- Dev Accuracy thus far: 85.000
Iter [4] Train: Time: 19.531s, Training Accuracy: 95.333             --- Dev Accuracy thus far: 85.333
Iter [5] Train: Time: 19.444s, Training Accuracy: 97.083             --- Dev Accuracy thus far: 86.000
Iter [6] Train: Time: 19.523s, Training Accuracy: 97.750             --- Dev Accuracy thus far: 85.333
Iter [7] Train: Time: 19.525s, Training Accuracy: 98.417             --- Dev Accuracy thus far: 85.333
Iter [8] Train: Time: 18.873s, Training Accuracy: 98.750             --- Dev Accura

In [23]:
t=[]
j=[]
l=[]
for pred in predicted:
    for i in pred:
        t.append(i)
for act in actual:
    for i in act:
        j.append(i)
for maxx in max_act:
    for i in maxx:
        l.append(i)
eval_metrics_2 = mx.metric.F1()
eval_metrics_2.update(labels = mx.nd.array(j), preds = mx.nd.array(t))
print("f1 score after using Xavier Initialization:",eval_metrics_2.get())

f1 score after using Xavier Initialization: ('f1', 0.3954802259887006)


#### FastText 

In [24]:
import fasttext
with open('listfile.txt', 'w') as filehandle:  
    for item in padded_sentences:
        filehandle.write(str(item) + "\n")
filehandle.close()
f1 = open('listfile.txt', 'r')
f2 = open('file2.txt', 'w')
for line in f1:
    f2.write(line.replace('[', '').replace(']','').replace(',','').replace("'", ""))

f1.close()
f2.close()

In [25]:
model = fasttext.skipgram('file2.txt', 'model', lr=0.1, dim=200, ws=5,word_ngrams=6)
model = fasttext.load_model('model.bin', encoding='utf-8')

In [26]:
embeddings_index = {}
with open('model.vec') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()
embedding_matrix = np.zeros((len(model.words ) + 1, model.dim))
i=0
for word in model.words :
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        i+=1


In [27]:
x_fast = np.array([
            [model[word] for word in sentence]
            for sentence in padded_sentences])

In [28]:
x_fast_train, y_fast_train, x_fast_test, y_fast_test= create_shuffle(x_fast,y)
sentence_size_fast = x_fast_train.shape[1]

In [29]:
batch_size = 20
print('batch size', batch_size)

input_x = mx.sym.Variable('data') # placeholder for input data
input_y = mx.sym.Variable('softmax_label') # placeholder for output label
num_embed = 200 # dimensions to embed words into
#print('embedding dimensions', num_embed)
#weight_matrix = mx.nd.array(embedding_matrix)
#the_emb_3 = mx.sym.Variable('weights')
#embed_layer = mx.sym.Embedding(data=input_x, input_dim=model.dim, output_dim=num_embed,weight=the_emb_3,name='vocab_embed')
# reshape embedded data for next layer
#conv_input = input_x
conv_input = mx.sym.Reshape(data=input_x, shape=(batch_size, 1, sentence_size_fast, num_embed))
filter_list=[2,3, 4, 5] # the size of filters to use
print('convolution filters', filter_list)

num_filter=100
pooled_outputs = []
for filter_size in filter_list:
    convi = mx.sym.Convolution(data=conv_input, kernel=(filter_size, num_embed), num_filter=num_filter)
    relui = mx.sym.Activation(data=convi, act_type='relu')
    pooli = mx.sym.Pooling(data=relui, pool_type='max', kernel=(sentence_size_fast - filter_size + 1, 1), stride=(1, 1))
    pooled_outputs.append(pooli)

# combine all pooled outputs
total_filters = num_filter * len(filter_list)
concat = mx.sym.Concat(*pooled_outputs, dim=1)

# reshape for next layer
h_pool = mx.sym.Reshape(data=concat, shape=(batch_size, total_filters))

num_label = 2

cls_weight = mx.sym.Variable('cls_weight')
cls_bias = mx.sym.Variable('cls_bias')

fc = mx.sym.FullyConnected(data=h_pool, weight=cls_weight, bias=cls_bias, num_hidden=num_label)

# softmax output
sm = mx.sym.SoftmaxOutput(data=fc, label=input_y, name='softmax')

# set CNN pointer to the "back" of the network
cnn = sm


batch size 20
convolution filters [2, 3, 4, 5]


In [30]:
from collections import namedtuple
import math
import time

# Define the structure of our CNN Model (as a named tuple)
CNNModel = namedtuple("CNNModel", ['cnn_exec', 'symbol', 'data', 'label', 'param_blocks'])

# Define what device to train/test on, use GPU if available
ctx = mx.cpu()
arg_names = cnn.list_arguments()

input_shapes = {}
input_shapes['data'] = (batch_size, sentence_size_fast,num_embed)

arg_shape, out_shape, aux_shape = cnn.infer_shape(**input_shapes)
arg_arrays = [mx.nd.zeros(s, ctx) for s in arg_shape]
args_grad = {}
for shape, name in zip(arg_shape, arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    args_grad[name] = mx.nd.zeros(shape, ctx)

cnn_exec = cnn.bind(ctx=ctx, args=arg_arrays, args_grad=args_grad, grad_req='add')

param_blocks = []
arg_dict = dict(zip(arg_names, cnn_exec.arg_arrays))
initializer = mx.initializer.Uniform(0.1)
for i, name in enumerate(arg_names):
    if name in ['softmax_label', 'data']: # input, output
        continue
    initializer(mx.init.InitDesc(name), arg_dict[name])

    param_blocks.append( (i, arg_dict[name], args_grad[name], name) )

data = cnn_exec.arg_dict['data']
label = cnn_exec.arg_dict['softmax_label']
fixed_param_names =['weights']
#weights=weight_matrix
cnn_model= CNNModel(cnn_exec=cnn_exec, symbol=cnn, data=data, label=label, param_blocks=param_blocks)

In [31]:
optimizer = 'rmsprop'
max_grad_norm = 0.01
learning_rate = 0.005
epoch = 10
predicted=[]
actual=[]
max_act=[]
print('optimizer', optimizer)
print('maximum gradient', max_grad_norm)
print('learning rate (step size)', learning_rate)
print('epochs to train for', epoch)

# create optimizer
opt = mx.optimizer.create(optimizer)
opt.lr = learning_rate

updater = mx.optimizer.get_updater(opt)

# For each training epoch
for iteration in range(epoch):
    tic = time.time()
    num_correct = 0
    num_total = 0

    # Over each batch of training data
    for begin in range(0, x_fast_train.shape[0], batch_size):
        batchX = x_fast_train[begin:begin+batch_size]
        batchY = y_fast_train[begin:begin+batch_size]
        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.label[:] = batchY

        # forward
        cnn_model.cnn_exec.forward(is_train=False)

        # backward
        cnn_model.cnn_exec.backward()

        # eval on training data
        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        

        # update weights
        norm = 0
        for idx, weight, grad, name in cnn_model.param_blocks:
            grad /= batch_size
            l2_norm = mx.nd.norm(grad).asscalar()
            norm += l2_norm * l2_norm

        norm = math.sqrt(norm)
        for idx, weight, grad, name in cnn_model.param_blocks:
            if norm > max_grad_norm:
                grad *= (max_grad_norm / norm)

            updater(idx, grad, weight)

            # reset gradient to zero
            grad[:] = 0.0

    # Decay learning rate for this epoch to ensure we are not "overshooting" optima
    if iteration % 50 == 0 and iteration > 0:
        opt.lr *= 0.5
        print('reset learning rate to %g' % opt.lr)

    # End of training loop for this epoch
    toc = time.time()
    train_time = toc - tic
    train_acc = num_correct * 100 / float(num_total)

    # Saving checkpoint to disk
    if (iteration + 1) % 10 == 0:
        prefix = 'cnn'
        cnn_model.symbol.save('./%s-symbol.json' % prefix)
        save_dict = {('arg:%s' % k) : v  for k, v in cnn_model.cnn_exec.arg_dict.items()}
        save_dict.update({('aux:%s' % k) : v for k, v in cnn_model.cnn_exec.aux_dict.items()})
        param_name = './%s-%04d.params' % (prefix, iteration)
        mx.nd.save(param_name, save_dict)
        print('Saved checkpoint to %s' % param_name)


    # Evaluate model after this epoch on dev (test) set
    num_correct = 0
    num_total = 0

    # For each test batch
    for begin in range(0, x_test.shape[0], batch_size):
        batchX = x_fast_test[begin:begin+batch_size]
        batchY = y_fast_test[begin:begin+batch_size]

        if batchX.shape[0] != batch_size:
            continue

        cnn_model.data[:] = batchX
        cnn_model.cnn_exec.forward(is_train=False)

        num_correct += sum(batchY == np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))
        num_total += len(batchY)
        predicted.append(cnn_model.cnn_exec.outputs[0].asnumpy())
        actual.append(batchY)
        max_act.append(np.argmax(cnn_model.cnn_exec.outputs[0].asnumpy(), axis=1))

    dev_acc = num_correct * 100 / float(num_total)
    print('Iter [%d] Train: Time: %.3fs, Training Accuracy: %.3f \
            --- Dev Accuracy thus far: %.3f' % (iteration, train_time, train_acc, dev_acc))

optimizer rmsprop
maximum gradient 0.01
learning rate (step size) 0.005
epochs to train for 10
Iter [0] Train: Time: 10.677s, Training Accuracy: 84.917             --- Dev Accuracy thus far: 85.667
Iter [1] Train: Time: 10.271s, Training Accuracy: 85.500             --- Dev Accuracy thus far: 86.667
Iter [2] Train: Time: 10.378s, Training Accuracy: 87.417             --- Dev Accuracy thus far: 88.000
Iter [3] Train: Time: 7.079s, Training Accuracy: 89.417             --- Dev Accuracy thus far: 88.333
Iter [4] Train: Time: 7.095s, Training Accuracy: 90.583             --- Dev Accuracy thus far: 88.667
Iter [5] Train: Time: 7.128s, Training Accuracy: 91.500             --- Dev Accuracy thus far: 85.667
Iter [6] Train: Time: 6.839s, Training Accuracy: 91.500             --- Dev Accuracy thus far: 86.333
Iter [7] Train: Time: 6.738s, Training Accuracy: 92.917             --- Dev Accuracy thus far: 89.000
Iter [8] Train: Time: 7.159s, Training Accuracy: 91.583             --- Dev Accuracy t

In [32]:
t=[]
j=[]
l=[]
for pred in predicted:
    for i in pred:
        t.append(i)
for act in actual:
    for i in act:
        j.append(i)
for maxx in max_act:
    for i in maxx:
        l.append(i)
eval_metrics_2 = mx.metric.F1()
eval_metrics_2.update(labels = mx.nd.array(j), preds = mx.nd.array(t))
print("f1 score after using fasttext:",eval_metrics_2.get())

f1 score after using fasttext: ('f1', 0.5604395604395606)


#### f1-score improved after using fasttext because of the sub-word information provided to the model.